In [6]:
import sqlite3
import pandas as pd

# Connect to the Chinook SQLite database
conn = sqlite3.connect('/kaggle/input/chinook/Chinook_Sqlite.sqlite')

# Function to run query, save output CSV, and return DataFrame
def run_query_save_csv(query, csv_filename):
    df = pd.read_sql_query(query, conn)
    df.to_csv(csv_filename, index=False)
    print(f" Saved '{csv_filename}'")
    return df

# Top-selling tracks by quantity sold
query_top_tracks = """
SELECT
    t.Name AS TrackName,
    SUM(il.Quantity) AS TotalQuantitySold
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
GROUP BY t.Name
ORDER BY TotalQuantitySold DESC
LIMIT 10;
"""
df_top_tracks = run_query_save_csv(query_top_tracks, 'top_10_tracks_by_quantity.csv')

# Revenue per region 
query_revenue_region = """
SELECT
    BillingCountry,
    ROUND(SUM(Total), 2) AS TotalRevenue
FROM Invoice
GROUP BY BillingCountry
ORDER BY TotalRevenue DESC;
"""
df_revenue_region = run_query_save_csv(query_revenue_region, 'revenue_per_region.csv')

# Monthly sales performance
query_monthly_performance = """
SELECT
    strftime('%Y-%m', InvoiceDate) AS YearMonth,
    ROUND(SUM(Total), 2) AS MonthlyRevenue,
    COUNT(InvoiceId) AS NumberOfInvoices
FROM Invoice
GROUP BY YearMonth
ORDER BY YearMonth;
"""
df_monthly_performance = run_query_save_csv(query_monthly_performance, 'monthly_sales_performance.csv')

# Top 10 tracks ranked by revenue using window function RANK()
query_rank_tracks = """
SELECT
    TrackName,
    TotalRevenue,
    RANK() OVER (ORDER BY TotalRevenue DESC) AS RevenueRank
FROM (
    SELECT
        t.Name AS TrackName,
        SUM(il.UnitPrice * il.Quantity) AS TotalRevenue
    FROM InvoiceLine il
    JOIN Track t ON il.TrackId = t.TrackId
    GROUP BY t.Name
) AS TrackRevenue
ORDER BY RevenueRank
LIMIT 10;
"""
df_rank_tracks = run_query_save_csv(query_rank_tracks, 'top_10_tracks_ranked_by_revenue.csv')

conn.close()


 Saved 'top_10_tracks_by_quantity.csv'
 Saved 'revenue_per_region.csv'
 Saved 'monthly_sales_performance.csv'
 Saved 'top_10_tracks_ranked_by_revenue.csv'
